In [3]:
import json
import re
from pyspark.sql import *
from pyspark import SparkContext, SQLContext

In [5]:

sc = SparkContext()
#hdfs:///datasets/tweets-leon
text_file = sc.textFile("hdfs:///datasets/tweets-leon")



In [6]:
sample = text_file.take(10)

In [9]:
sample

['en\t345963923251539968\tSat Jun 15 18:00:01 +0000 2013\tLetataleta\tRT @silsilfani: the world is not a wish-granting machine. dont be surprised when everything always end up disappointing.',
 "en\t345963923297673217\tSat Jun 15 18:00:01 +0000 2013\tJamesonN7\tRT @WhosThisHoe: I'd rather sleep with a nice guy because they always finish last",
 "en\t345963923259924480\tSat Jun 15 18:00:01 +0000 2013\tLauraEllynJones\tCan't stand people who lie then blame it on someone else #boreoff",
 'it\t345963923276697601\tSat Jun 15 18:00:01 +0000 2013\tChialettaFClub\t@ChialettaFClub: #rt seguimi ti seguo ti voto il profilo\\nX9\\nOBBIETTIVO:300',
 'fr\t345963923255730176\tSat Jun 15 18:00:01 +0000 2013\t_irem61_\tRT @DHC_Music: Terrorism ... #FreePalestina http://t.co/OLWnVlW682',
 'en\t345963923247345664\tSat Jun 15 18:00:01 +0000 2013\tdannamaee\t@JennnnyDo better than Starbucks??',
 'en\t345963923272511489\tSat Jun 15 18:00:01 +0000 2013\tmalenpalmeyro1\tRT @arthurmeyer13: Thank you for the Fr